# Criação das tabelas dimensão e fatos

In [0]:
from pyspark.sql.functions import monotonically_increasing_id, row_number
from pyspark.sql import Window

In [0]:
df_goodreads = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/silver/goodreads')
df_kindle = spark.read.format('delta').load('abfss://livros@recomendador.dfs.core.windows.net/silver/kindle')

> Juntar as tabelas com as informações que queremos para análise

In [0]:
df_goodreads.printSchema()

In [0]:
df_kindle.printSchema()

In [0]:
df_join = df_goodreads.join(df_kindle, df_goodreads.title == df_kindle.title, 'inner') \
                    .drop(df_goodreads.title, df_goodreads.author, df_goodreads.rating, df_kindle.reviews)

## Exclusão das variáveis

In [0]:
del df_goodreads, df_kindle

In [0]:
df_join.groupBy(df_join.title) \
        .count() \
        .orderBy('count', ascending=False) \
        .show(10)

In [0]:
df_join.columns

## Criação das tabelas e dimensões

> Tabela _dimensão autor_

In [0]:
w = Window.orderBy(df_join.author).distinct()
df_dim_autor = df_join.select(row_number().over(w).alias('id'), df_join.author)

> Tabela _dimensão livro_

In [0]:
w = Window.orderBy(df_join.title).distinct()
df_dim_livro = df_join.select(row_number().over(w).alias('id'), df_join.isbn, df_join.title, df_join.bookformat)

> Tabelas _ponte grupo categoria_, _dimensão grupo_ e _dimensão categoria_

In [0]:
df_join.select(df_join.category_id, df_join.category_name, df_join.genre).toPandas().head(10)

In [0]:
del w

> Tabela _fatos_

In [0]:
df_fatos = df_join.select(df_join.id_livro, df_join.isbn, df_join.pages.alias('numero_de_paginas'), df_join.stars.alias('numero_de_estrelas'), df_join.reviews.alias('numero_de_avaliacoes'), df_join.totalratings.alias('total_de_avaliacoes'), df_join.price.alias('preco'), df_join.isKindleUnlimited.alias('esta_no_kindle_unlimited'), df_join.isGoodReadsChoice.alias('esta_no_goodreads_choice'), df_join.publishedDate.alias('data_de_publicacao'))

In [0]:
df_fatos.columns

## Salvação dos dados na camada _gold_

## Exclusão das variáveis

In [0]:
del df_join, df_goodreads, df_kindle